In [1]:
import torch
import sparseconvnet as scn
import tables as tb
import pandas as pd
import re
import numpy as np

In [2]:
from next_sparseconvnet.utils.data_loaders import DataGen, collatefn, LabelType

In [3]:
dataset_path = '/home/mperez/NEXT_SPARSECONVNET/next_sparseconvnet/test_files/MC_dataset.h5'

In [4]:
datagen = DataGen(dataset_path, LabelType.Segmentation)

In [5]:
with datagen:
    batch = [datagen[i] for i in range(3)]
coord, ener, lab, ev = collatefn(batch)

In [6]:
coord, ener, lab, ev = collatefn(batch)

In [7]:
dim = len(coord[0]) - 1

Pruebo la estructura del bloque residual con datos a ver que tal

In [10]:
inp     = scn.InputLayer(dim, (50, 50, 50)) #dimension is 3 (3d) and spatial_size must be greater than max(binsZ)/len(binsZ) = 550/11 = 50
bnr1    = scn.BatchNormReLU(1) #nPlanes = 1 = inplanes
conv1   = scn.Convolution(dim, 1, 2, 2, 2, False) #dim, nIn = inplanes, nOut = 2*inplanes, ker, str, bias
bnr2    = scn.BatchNormReLU(2) #tras lo anterior, nPlanes supuestamente se duplicó de 1 a 2
subconv = scn.SubmanifoldConvolution(dim, 2, 2, 2, False) #dim, nIn = 2*inplanes, nOut = 2*inplanes(aqui creo que no cambio N), ker, bias, groups

conv2   = scn.Convolution(dim, 1, 2, 2, 2, False)

add     = scn.AddTable()

x = inp((coord, ener))
print(x.spatial_size, x.features.shape)
x = bnr1(x)
print(x.spatial_size, x.features.shape[1])


#f1
y1 = conv1(x)
print(y1.spatial_size, y1.features.shape[1])
y1 = bnr2(y1)
print(y1.spatial_size, y1.features.shape[1])
y1 = subconv(y1)
print(y1.spatial_size, y1.features.shape[1])

#f2
y2 = conv2(x)
print(y2.spatial_size, y2.features.shape[1])


#sum
out = add([y1, y2])
print(y2.spatial_size, y2.features.shape[1])

#print(out)

tensor([50, 50, 50]) torch.Size([333, 1])
tensor([50, 50, 50]) 1
tensor([25, 25, 25]) 2
tensor([25, 25, 25]) 2
tensor([25, 25, 25]) 2
tensor([25, 25, 25]) 2
tensor([25, 25, 25]) 2


In [20]:
class ResidualBlock_downsample(torch.nn.Module):
    def __init__(self, inplanes, kernel, stride, bias = False, dim = 3):
        torch.nn.Module.__init__(self)
        
        #f1
        self.bnr1    = scn.BatchNormReLU(inplanes) 
        self.conv1   = scn.Convolution(dim, inplanes, 2 * inplanes, kernel, stride, bias)
        self.bnr2    = scn.BatchNormReLU(2 * inplanes) 
        self.subconv = scn.SubmanifoldConvolution(dim, 2 * inplanes, 2 * inplanes, kernel, bias)
        
        #f2
        self.conv2   = scn.Convolution(dim, inplanes, 2 * inplanes, kernel, stride, bias) 
        
        self.add     = scn.AddTable()
        
    def forward(self, x):  
        x = self.bnr1(x)
        
        #f1
        y1 = self.conv1(x)
        y1 = self.bnr2(y1)
        y1 = self.subconv(y1)
        
        #f2
        y2 = self.conv2(x)
        
        #sum
        out = self.add([y1, y2])
        
        return out

In [21]:
block = ResidualBlock_downsample(1, 2, 2, False) #pruebo con kernel = 2 porque para mi spatial_size = (50, 50, 50) con 3 da error, supongo que tendría que poner un spatial size por ejemplo de (51, 51, 51) ya que el mínimo es 50

In [15]:
out = block.forward(x)

In [19]:
print(out.features.shape)

torch.Size([17, 2])


In [9]:
from next_sparseconvnet.networks.building_blocks import *